# Tutorial 06 - Building RESTQL APIs

Query as API (aka RESTQL) enables developers to quickly convert saved C8QL queries into geo-distributed REST APIs. This eliminates the need for separate backend servers & containers for CRUD operations.

## Step 06-A - Import Libraries

In [ ]:
import json

## Step 06-B - Import c8Client

In [ ]:
from c8 import C8Client

## Step 06-C - Define Login Credentials


In [ ]:
fed_url = "verizon.5g.macrometa.io"
guest_mail = "email"
guest_password = "password"
geo_fabric = "_system"

## Step 06-D - Connect to GDN

Now that you have added your login credentials, lets connect to GDN

In [ ]:
print("\n ------- CONNECTION SETUP  ------")
print("tenant: {}, geofabric:{}".format(guest_mail, geo_fabric))
client = C8Client(protocol='https', host=fed_url, port=443,
                email=guest_mail, password=guest_password,
                geofabric=geo_fabric)    
    

## Step 06-E - Create a Geo Replicated Collection

Before we create the RESTQL APIs lets create a collection that we can work with

In [ ]:
collection_name = "person"

print("\n ------- CREATE GEO-REPLICATED COLLECTION  ------")
if client.has_collection(collection_name):
    print("Collection exists")
else:
    employees = client.create_collection(collection_name)
    
print("Created collection: {}".format(collection_name))

## Step 06-F - Create Rest APIs (RESTQLs)

### Step 06-F1 - Defining our Queries

In this section we are going to define five RESTQL Query APIs

In [ ]:
value = "INSERT {'firstname':@firstname, 'lastname':@lastname, 'email':@email, 'zipcode':@zipcode, '_key': 'abc'} IN %s" % collection_name
parameter = {"firstname": "", "lastname": "", "email": "", "zipcode": ""}

insert_data = {"query": {"name": "insertRecord", "parameter": parameter, "value": value}} 
get_data = {"query": {"name": "getRecords", "value": "FOR doc IN %s RETURN doc" % collection_name}}
update_data = {"query": {"name": "updateRecord", "value": "UPDATE 'abc' WITH { \"lastname\": \"cena\" } IN %s" % collection_name }}
delete_data= {"query": {"name": "deleteRecord", "value": "REMOVE 'abc' IN %s" % collection_name}}
get_count = {"query": {"name": "countRecords", "value": "RETURN COUNT(FOR doc IN %s RETURN 1)" % collection_name}}

### Step 06-F1 - Creating the RESTQL APIs

The Following Code will create a set of RESTQL APIs in GDN - once complete you can check that they exist from your console access, under Queries, and navigate to "Saved Queries"

In [ ]:
print("\n ------- CREATE RESTQLs  ------")
client.create_restql(insert_data)  # name: insertRecord
client.create_restql(get_data)  # name: getRecords
client.create_restql(update_data)  # name: updateRecord
client.create_restql(delete_data)  # name: deleteRecord
client.create_restql(get_count)  # name: countRecords
  
print(json.dumps(client.get_restqls(), indent=4))  

## Step 06-G - Execute RESTQLs

**Note: we are going to skip testing the "deleteRecord" API in this section - so that we can demonstrate how to update a RESTQL Query API first.**

### Step 06-G1 - Test the "insertRecord" RESTQL

In [ ]:
print("\n ------- EXECUTE RESTQLs ------")
print("Insert data....")
response = client.execute_restql(
    "insertRecord",
    {"bindVars": {"firstname": "john", "lastname": "doe",
                    "email": "john.doe@macrometa.io", "zipcode": "511037"}})

print("Get data....")
response = client.execute_restql("getRecords")

print(json.dumps(response, indent=4))  


### Step 06-G2 - Test the "updateRecord" RESTQL

In [ ]:
print("Update data....")
response = client.execute_restql("updateRecord")

print(json.dumps(response, indent=4))  

### Step 06-G3 - Test the "getRecords" RESTQL

In [ ]:
print("Get data....")
response = client.execute_restql("getRecords")

print(json.dumps(response, indent=4))  

### Step 06-G4 - Test the "countRecords" RESTQL

In [ ]:
print("Count records....")
response = client.execute_restql("countRecords")

print(json.dumps(response, indent=4))  

## Step 06-H - Updating a RESTQL API

### Step 06-H1 - Test that we can query GDN and see the document

A common requirment when building an application is to make a change to an API and update it. 

To demonstrate this, lets first query the database using the "execute_query" method we learnt in an earlier section

Once run, we should see the document in the cell output in JSON format so we can read it easily.

In [ ]:
print("Get docs in \"person\" collection")

cursor = client.execute_query('FOR doc IN person RETURN doc')

docs = [document for document in cursor]

print(json.dumps(docs, indent=4))

### Step 06-H2 - Updating a RESTQL API

The follow cell show us how to add a new parameter to the Query, in this example we are going to add a the parameter "phone" to the query.

In [ ]:
value = f"INSERT {{'firstname':@firstname, 'lastname':@lastname, 'email':@email, 'zipcode':@zipcode, 'phone': @phone}} IN {collection_name}"
parameter = {"firstname": "", "lastname": "", "email": "", "zipcode": "", "phone": ""}

insert_data = { "query": { "parameter": parameter, "value": value } } 

query_name = "insertRecord"
response = client.update_restql(query_name, insert_data)
print(response)



### Step 06-H3 - Testing the updated RESTQL API

Now that we have updated the Query, lets add a new record using it to test that it works.

In [ ]:
print("Insert data with updated query worker")
response = client.execute_restql(
    query_name,
    {"bindVars": {"firstname": "john", "lastname": "doe",
                    "email": "john.doe@macrometa.io", "zipcode": "511037", "phone": "213-555-9578"}})
print(response)

### Step 06-H4 - Checking the data in the collection

Now lets check that the data has been updated in the record by using the "execute_query" method - we should see two documents. 

1. the original document without a phone number, and
2. a second document, but now with a phone numnber added

**Hint: If you would like to return to "Step 06-G4 - Test the "countRecords" RESTQL" and re run the code, you should see the count value increase by 1**

In [ ]:
print("Get docs in \"person\" collection, after executing the updated query")

cursor = client.execute_query('FOR doc IN person RETURN doc')

docs = [document for document in cursor]

print(json.dumps(docs, indent=4))

## Step 06-I - Deleting a record using a RESTQL API

In [ ]:
print("Delete data....")
response = client.execute_restql("deleteRecord")

print(json.dumps(response, indent=4))  

## Step 06-J - Delete RESTQLs

We might want to delete our RESQL APIs completely.

For each RESTQL we want to remove we use the "delete_restql" method as shown in the cell below

In [ ]:
print("\n ------- DELETE RESTQLs ------")
client.delete_restql("insertRecord")
client.delete_restql("getRecords")
client.delete_restql("updateRecord")
client.delete_restql("countRecords")
client.delete_restql("deleteRecord")

print("\n ------- DONE  ------")

## Step 06-K - Delete the Collection

Now let us remove the collection we added during this Tutorial

In [ ]:
print("Collection Deleted: ",client.delete_collection(collection_name))

## Section Completed!

Congratulations!, another tutorial completed.

**Note: To use the created RESTQL APIs you should use the API reference documentation for detailed insutructions.**

e.g. /_fabric/{fabric}/_api/restql/{name} 

where {name} is the name of your RESTQL API e.g. "insertRecord"

